In [3]:
#!/usr/bin/env python
# coding: utf-8

# Runs

# In[236]:


import pandas as pd, numpy as np, matplotlib, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import preprocessing
%matplotlib inline


# In[237]:


allData = pd.read_csv('all-np.csv')


# In[238]:


# Some preprocessing on the column names
allData.astype({'Season': 'category'}).dtypes # set season to category instead of int
allData.set_index(['playerid', 'Name'])

# drop NaN values, which only come into play for pitchers running the bases (a very atypical occurrence)
allData = allData.dropna()

# sort all the rows by player ID first, then season. Makes it easier to figure out which player played which seasons
allData = allData.sort_values(by=['playerid', 'Season'])

# scale all relevant stats
allData[['OBP', 'PA', 'wSB', 'UBR']] = preprocessing.scale(allData[['OBP', 'PA', 'wSB', 'UBR']])

# assign X and Y sets for regression
X = allData[['OBP', 'PA', 'wSB', 'UBR']].values
Y = allData[['R']].values

# dictionary assigning number of years in our data set played to each player
playerYears = dict() # dict of {'player ID': 'years in our data set that they played (1, 2, or 3)'}
pid, yearsPlayed = np.unique(allData['playerid'], return_counts=True)
for i in range(len(pid)):
    playerYears[pid[i]] = yearsPlayed[i]

# dictionary assigning names to player IDs
playerNames = dict() # dict where key is player ID, value is player name
# Assign each player ID to a name for future purposes
for i, r in allData.iterrows():
    playerNames[r['playerid']] = r['Name']

# assigns weights to each player and the data for that year. If played 2016-18, keep the weights of 0.6, 0.3, 0.1. If played two years, 0.7 and 0.3. If only played one year, assign 1 for now
for index, row in allData.iterrows():
    if playerYears[row['playerid']] == 3:
        continue
    elif playerYears[row['playerid']] == 2: # played in 16-17 or 17-18
        played1617 = False
        if row['Season'] == 2016:
            played1617 = True
            allData.set_value(index, 'weight', 0.3)
        elif row['Season'] == 2017:
            if played1617:
                allData.set_value(index, 'weight', 0.7)
            else:
                allData.set_value(index, 'weight', 0.3)
        else:
            allData.set_value(index, 'weight', 0.7)
    else:
        allData.set_value(index, 'weight', 1)

# add newly calculated weighted stats to our dataframe as columns
allData['weightOBP'] = allData['weight']*allData['OBP']
allData['weightPA'] = allData['weight']*allData['PA']
allData['weightSB'] = allData['weight']*allData['wSB']
allData['weightUBR'] = allData['weight']*allData['UBR']

# sum up weighted averages by player, so that all years are combined
weightedRunStats2019 = allData.groupby('playerid', as_index=False).sum()
weightedRunStats2019 = weightedRunStats2019[['playerid', 'weightOBP', 'weightPA', 'weightSB', 'weightUBR']]

# we lost player names when doing the group by and sum, so get the player names and put it back in our new data frame
names = []
for i in pid:
    names.append(playerNames[i])
weightedRunStats2019['Name'] = names
weightedRunStats2019['Season'] = 2019

# move name and season columns to the front of the dataframe
cols = weightedRunStats2019.columns.tolist()
cols = cols[-2:] + cols[:-2]
weightedRunStats2019 = weightedRunStats2019[cols]


# In[239]:


# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
regressor = LinearRegression()
regressor.fit(X, Y)


# In[240]:


X_2019 = weightedRunStats2019[['weightOBP', 'weightPA', 'weightSB', 'weightUBR']].values
y_pred = regressor.predict(X_2019)


# In[241]:


y_pred_list = [] # list of y_pred so we can add it to a dataframe
for i in range(len(y_pred)):
    y_pred_list.append(y_pred[i][0])

weightedRunStats2019['runsPredicted'] = y_pred_list
weightedRunStats2019 = weightedRunStats2019.sort_values(by=['runsPredicted'], ascending=False)

c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:76: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:67: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:72: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:74: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [4]:
allData[['BABIP', 'Hard%', 'Med%', 'FB%+', 'LD+%', 'UBR', 'K%']] = preprocessing.scale(allData[['BABIP', 'Hard%', 'Med%', 'FB%+', 'LD+%', 'UBR', 'K%']])

# assign X and Y sets for regression
X = allData[['BABIP', 'Hard%', 'Med%', 'FB%+', 'LD+%', 'UBR', 'K%']].values
Y = allData[['AVG']].values

# dictionary assigning number of years in our data set played to each player
playerYears = dict() # dict of {'player ID': 'years in our data set that they played (1, 2, or 3)'}
pid, yearsPlayed = np.unique(allData['playerid'], return_counts=True)
for i in range(len(pid)):
    playerYears[pid[i]] = yearsPlayed[i]

# dictionary assigning names to player IDs
playerNames = dict() # dict where key is player ID, value is player name
# Assign each player ID to a name for future purposes
for i, r in allData.iterrows():
    playerNames[r['playerid']] = r['Name']

# assigns weights to each player and the data for that year. If played 2016-18, keep the weights of 0.6, 0.3, 0.1. If played two years, 0.7 and 0.3. If only played one year, assign 1 for now
for index, row in allData.iterrows():
    if playerYears[row['playerid']] == 3:
        continue
    elif playerYears[row['playerid']] == 2: # played in 16-17 or 17-18
        played1617 = False
        if row['Season'] == 2016:
            played1617 = True
            allData.set_value(index, 'weight', 0.3)
        elif row['Season'] == 2017:
            if played1617:
                allData.set_value(index, 'weight', 0.7)
            else:
                allData.set_value(index, 'weight', 0.3)
        else:
            allData.set_value(index, 'weight', 0.7)
    else:
        allData.set_value(index, 'weight', 1)

'BABIP', 'Hard%', 'Med%', 'FB%+', 'LD+%', 'UBR', 'K%'
# add newly calculated weighted stats to our dataframe as columns
allData['weightBABIP'] = allData['weight']*allData['BABIP']
allData['weightHH'] = allData['weight']*allData['Hard%']
allData['weightMed'] = allData['weight']*allData['Med%']
allData['weightFB'] = allData['weight']*allData['FB%+']
allData['weightLD'] = allData['weight']*allData['LD+%']
allData['weightUBR'] = allData['weight']*allData['UBR']
allData['weightK'] = allData['weight']*allData['K%']

# sum up weighted averages by player, so that all years are combined
weightedAvgStats2019 = allData.groupby('playerid', as_index=False).sum()
weightedAvgStats2019 = weightedAvgStats2019[['playerid', 'weightBABIP', 'weightHH', 'weightMed', 'weightFB', 'weightLD', 'weightUBR', 'weightK']]

# we lost player names when doing the group by and sum, so get the player names and put it back in our new data frame
names = []
for i in pid:
    names.append(playerNames[i])
weightedAvgStats2019['Name'] = names
weightedAvgStats2019['Season'] = 2019

# move name and season columns to the front of the dataframe
cols = weightedAvgStats2019.columns.tolist()
cols = cols[-2:] + cols[:-2]
weightedAvgStats2019 = weightedAvgStats2019[cols]


# In[13]:


# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
regressor = LinearRegression()
regressor.fit(X, Y)

# In[14]:


X_2019 = weightedAvgStats2019[['weightBABIP', 'weightHH', 'weightMed', 'weightFB', 'weightLD', 'weightUBR', 'weightK']].values
y_pred = regressor.predict(X_2019)


# In[15]:


y_pred_list = [] # list of y_pred so we can add it to a dataframe
for i in range(len(y_pred)):
    y_pred_list.append(y_pred[i][0])

weightedAvgStats2019['AVGPredicted'] = y_pred_list
weightedAvgStats2019 = weightedAvgStats2019.sort_values(by=['AVGPredicted'], ascending=False)
print(weightedAvgStats2019)

c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:36: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


                      Name  Season  playerid  weightBABIP  weightHH  \
454            Jeff McNeil    2019     15362     1.604640 -0.532835   
132            Jose Altuve    2019      5417     1.546836 -0.268916   
119          Justin Turner    2019      5235     0.737581  1.469365   
399           Mookie Betts    2019     13611     0.899983  1.185896   
396           Jesse Winker    2019     13590     0.971549  1.699072   
..                     ...     ...       ...          ...       ...   
431         Pedro Severino    2019     14523    -2.469159 -1.836137   
497          Byung-ho Park    2019     18497    -1.946172  0.574973   
230         Chris Herrmann    2019      9284    -2.579262  0.151400   
138  Jarrod Saltalamacchia    2019      5557    -2.166377  0.640138   
32             Ryan Howard    2019      2154    -2.634313  2.024898   

     weightMed  weightFB  weightLD  weightUBR   weightK  AVGPredicted  
454  -0.108276  0.711006 -0.066205   1.691304 -1.900313      0.338804  
132

In [5]:
# scale relevants stats
allData[['FB%+', 'Hard%', 'Pull%', 'SLG', 'PA']] = preprocessing.scale(allData[['FB%+', 'Hard%', 'Pull%', 'SLG', 'PA']])

# assign X and Y sets for regression
X = allData[['FB%+', 'Hard%', 'Pull%', 'SLG', 'PA']].values
Y = allData[['HR']].values

# dictionary assigning number of years in our data set played to each player
playerYears = dict() # dict of {'player ID': 'years in our data set that they played (1, 2, or 3)'}
pid, yearsPlayed = np.unique(allData['playerid'], return_counts=True)
for i in range(len(pid)):
    playerYears[pid[i]] = yearsPlayed[i]

# dictionary assigning names to player IDs
playerNames = dict() # dict where key is player ID, value is player name
# Assign each player ID to a name for future purposes
for i, r in allData.iterrows():
    playerNames[r['playerid']] = r['Name']

# assigns weights to each player and the data for that year. If played 2016-18, keep the weights of 0.6, 0.3, 0.1. If played two years, 0.7 and 0.3. If only played one year, assign 1 for now
for index, row in allData.iterrows():
    if playerYears[row['playerid']] == 3:
        continue
    elif playerYears[row['playerid']] == 2: # played in 16-17 or 17-18
        played1617 = False
        if row['Season'] == 2016:
            played1617 = True
            allData.set_value(index, 'weight', 0.3)
        elif row['Season'] == 2017:
            if played1617:
                allData.set_value(index, 'weight', 0.7)
            else:
                allData.set_value(index, 'weight', 0.3)
        else:
            allData.set_value(index, 'weight', 0.7)
    else:
        allData.set_value(index, 'weight', 1)

# add newly calculated weighted stats to our dataframe as columns
allData['weightFB'] = allData['weight']*allData['FB%+']
allData['weightPA'] = allData['weight']*allData['PA']
allData['weightHH'] = allData['weight']*allData['Hard%']
allData['weightPull'] = allData['weight']*allData['Pull%']
allData['weightSLG'] = allData['weight']*allData['SLG']

# sum up weighted averages by player, so that all years are combined
weightedHRStats2019 = allData.groupby('playerid', as_index=False).sum()
weightedHRStats2019 = weightedHRStats2019[['playerid', 'weightFB', 'weightPA', 'weightHH', 'weightPull', 'weightSLG']]

# we lost player names when doing the group by and sum, so get the player names and put it back in our new data frame
names = []
for i in pid:
    names.append(playerNames[i])
weightedHRStats2019['Name'] = names
weightedHRStats2019['Season'] = 2019

# move name and season columns to the front of the dataframe
cols = weightedHRStats2019.columns.tolist()
cols = cols[-2:] + cols[:-2]
weightedHRStats2019 = weightedHRStats2019[cols]


# In[13]:


# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
regressor = LinearRegression()
regressor.fit(X, Y)


# In[14]:


X_2019 = weightedHRStats2019[['weightFB', 'weightPA', 'weightHH', 'weightPull', 'weightSLG']].values
y_pred = regressor.predict(X_2019)

# In[15]:


y_pred_list = [] # list of y_pred so we can add it to a dataframe
for i in range(len(y_pred)):
    y_pred_list.append(y_pred[i][0])

weightedHRStats2019['HRsPredicted'] = y_pred_list
weightedHRStats2019 = weightedHRStats2019.sort_values(by=['HRsPredicted'], ascending=False)
print(weightedHRStats2019)

c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:35: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


                Name  Season  playerid  weightFB  weightPA  weightHH  \
4        David Ortiz    2019       745  1.387968  1.147429  2.024898   
473     Rhys Hoskins    2019     16472  2.169079  0.484159  0.729740   
399     Mookie Betts    2019     13611  1.023450  1.339011  1.185896   
389        Max Muncy    2019     13301  1.492116  0.189519  2.269267   
395     Jose Ramirez    2019     13510  0.945339  1.465191  0.174207   
..               ...     ...       ...       ...       ...       ...   
249    Ivan De Jesus    2019      9886 -1.424031 -1.382774 -1.689516   
362      Billy Burns    2019     12701 -1.163661 -0.794816 -3.465266   
468    Austin Slater    2019     16153 -2.830031 -1.501687  0.640138   
306       Tyler Holt    2019     11391 -1.371957 -1.613994 -1.950177   
376  Kelby Tomlinson    2019     13005 -1.632327 -1.521506 -2.259711   

     weightPull  weightSLG  HRsPredicted  
4      1.355486   2.792856     38.725419  
473    1.672406   1.539667     34.195002  
399   

In [6]:
# scale all relevant stats
allData[['BB%+', 'Hard%', 'LD+%', 'FB%+', 'Soft%', 'PA']] = preprocessing.scale(allData[['BB%+', 'Hard%', 'LD+%', 'FB%+', 'Soft%', 'PA']])

# assign X and Y sets for regression
X = allData[['BB%+', 'Hard%', 'LD+%', 'FB%+', 'Soft%', 'PA']].values
Y = allData[['OPS']].values

# dictionary assigning number of years in our data set played to each player
playerYears = dict() # dict of {'player ID': 'years in our data set that they played (1, 2, or 3)'}
pid, yearsPlayed = np.unique(allData['playerid'], return_counts=True)
for i in range(len(pid)):
    playerYears[pid[i]] = yearsPlayed[i]

# dictionary assigning names to player IDs
playerNames = dict() # dict where key is player ID, value is player name
# Assign each player ID to a name for future purposes
for i, r in allData.iterrows():
    playerNames[r['playerid']] = r['Name']
    
# assigns weights to each player and the data for that year. If played 2016-18, keep the weights of 0.6, 0.3, 0.1. If played two years, 0.7 and 0.3. If only played one year, assign 1 for now
for index, row in allData.iterrows():
    if playerYears[row['playerid']] == 3:
        continue
    elif playerYears[row['playerid']] == 2: # played in 16-17 or 17-18
        played1617 = False
        if row['Season'] == 2016:
            played1617 = True
            allData.set_value(index, 'weight', 0.3)
        elif row['Season'] == 2017:
            if played1617:
                allData.set_value(index, 'weight', 0.7)
            else:
                allData.set_value(index, 'weight', 0.3)
        else:
            allData.set_value(index, 'weight', 0.7)
    else:
        allData.set_value(index, 'weight', 1)

# add newly calculated weighted stats to our dataframe as columns
allData['weightBB'] = allData['weight']*allData['BB%+']
allData['weightHH'] = allData['weight']*allData['Hard%']
allData['weightLD'] = allData['weight']*allData['LD+%']
allData['weightFB'] = allData['weight']*allData['FB%+']
allData['weightSH'] = allData['weight']*allData['Soft%']
allData['weightPA'] = allData['weight']*allData['PA']

# sum up weighted averages by player, so that all years are combined
weightedOPSStats2019 = allData.groupby('playerid', as_index=False).sum()
weightedOPSStats2019 = weightedOPSStats2019[['playerid', 'weightBB', 'weightHH', 'weightLD', 'weightFB', 'weightSH', 'weightPA']]

# we lost player names when doing the group by and sum, so get the player names and put it back in our new data frame
names = []
for i in pid:
    names.append(playerNames[i])
weightedOPSStats2019['Name'] = names
weightedOPSStats2019['Season'] = 2019

# move name and season columns to the front of the dataframe
cols = weightedOPSStats2019.columns.tolist()
cols = cols[-2:] + cols[:-2]
weightedOPSStats2019 = weightedOPSStats2019[cols]


# In[239]:


# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
regressor = LinearRegression()
regressor.fit(X, Y)

# In[240]:


X_2019 = weightedOPSStats2019[['weightBB', 'weightHH', 'weightLD', 'weightFB', 'weightSH', 'weightPA']].values
y_pred = regressor.predict(X_2019)


# In[241]:


y_pred_list = [] # list of y_pred so we can add it to a dataframe
for i in range(len(y_pred)):
    y_pred_list.append(y_pred[i][0])

weightedOPSStats2019['opsPredicted'] = y_pred_list
weightedOPSStats2019 = weightedOPSStats2019.sort_values(by=['opsPredicted'], ascending=False)
print(weightedOPSStats2019)

c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:35: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


                Name  Season  playerid  weightBB  weightHH  weightLD  \
199   Matt Carpenter    2019      8090  2.093949  2.081917  1.212762   
263       Mike Trout    2019     10155  3.657305  1.438411  0.251880   
461      Aaron Judge    2019     15640  2.633514  2.246459  0.622979   
87        Joey Votto    2019      4314  2.694388  0.959447  2.173645   
4        David Ortiz    2019       745  1.626326  2.024898  0.530205   
..               ...     ...       ...       ...       ...       ...   
346      Kevan Smith    2019     12284 -1.777084 -1.770972 -0.265008   
271    Rafael Ortega    2019     10323 -0.559604 -1.901303 -0.927686   
165         JB Shuck    2019      6677 -1.057664 -1.917594 -1.457828   
307  Ronald Torreyes    2019     11400 -1.694074 -2.618119 -0.993953   
362      Billy Burns    2019     12701 -1.721744 -3.465266 -0.463811   

     weightFB  weightSH  weightPA  opsPredicted  
199  1.872257 -2.132872  1.302016      0.927421  
263  1.106768 -0.459647  0.876572  

In [7]:
# scale relevants stats
allData[['AVG', 'SF',  'SLG', 'PA', 'AB']] = preprocessing.scale(allData[['AVG', 'SF', 'SLG', 'PA', 'AB']])

# assign X and Y sets for regression
X = allData[['AVG', 'SF', 'SLG', 'PA', 'AB']].values
Y = allData[['RBI']].values

# dictionary assigning number of years in our data set played to each player
playerYears = dict() # dict of {'player ID': 'years in our data set that they played (1, 2, or 3)'}
pid, yearsPlayed = np.unique(allData['playerid'], return_counts=True)
for i in range(len(pid)):
    playerYears[pid[i]] = yearsPlayed[i]

# dictionary assigning names to player IDs
playerNames = dict() # dict where key is player ID, value is player name
# Assign each player ID to a name for future purposes
for i, r in allData.iterrows():
    playerNames[r['playerid']] = r['Name']
    
# assigns weights to each player and the data for that year. If played 2016-18, keep the weights of 0.6, 0.3, 0.1. If played two years, 0.7 and 0.3. If only played one year, assign 1 for now
for index, row in allData.iterrows():
    if playerYears[row['playerid']] == 3:
        continue
    elif playerYears[row['playerid']] == 2: # played in 16-17 or 17-18
        played1617 = False
        if row['Season'] == 2016:
            played1617 = True
            allData.set_value(index, 'weight', 0.3)
        elif row['Season'] == 2017:
            if played1617:
                allData.set_value(index, 'weight', 0.7)
            else:
                allData.set_value(index, 'weight', 0.3)
        else:
            allData.set_value(index, 'weight', 0.7)
    else:
        allData.set_value(index, 'weight', 1)

# add newly calculated weighted stats to our dataframe as columns
allData['weightAVG'] = allData['weight']*allData['AVG']
allData['weightPA'] = allData['weight']*allData['PA']
allData['weightSF'] = allData['weight']*allData['SF']
allData['weightSLG'] = allData['weight']*allData['SLG']
allData['weightAB'] = allData['weight']*allData['AB']

# sum up weighted averages by player, so that all years are combined
weightedRBIStats2019 = allData.groupby('playerid', as_index=False).sum()
weightedRBIStats2019 = weightedRBIStats2019[['playerid', 'weightAVG', 'weightPA', 'weightSF', 'weightSLG' ,'weightAB']]

# we lost player names when doing the group by and sum, so get the player names and put it back in our new data frame
names = []
for i in pid:
    names.append(playerNames[i])
weightedRBIStats2019['Name'] = names
weightedRBIStats2019['Season'] = 2019

# move name and season columns to the front of the dataframe
cols = weightedRBIStats2019.columns.tolist()
cols = cols[-2:] + cols[:-2]
weightedRBIStats2019 = weightedRBIStats2019[cols]


# In[13]:


# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
regressor = LinearRegression()
regressor.fit(X, Y)

# In[14]:


X_2019 = weightedRBIStats2019[['weightAVG', 'weightPA', 'weightSF', 'weightSLG', 'weightAB']].values
y_pred = regressor.predict(X_2019)


# In[15]:


y_pred_list = [] # list of y_pred so we can add it to a dataframe
for i in range(len(y_pred)):
    y_pred_list.append(y_pred[i][0])

weightedRBIStats2019['RBIsPredicted'] = y_pred_list
weightedRBIStats2019 = weightedRBIStats2019.sort_values(by=['RBIsPredicted'], ascending=False)
print(weightedRBIStats2019)

c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\edyua\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:35: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


                  Name  Season  playerid  weightAVG  weightPA  weightSF  \
107  Giancarlo Stanton    2019      4949   0.361599  1.488313  1.825024   
220        Khris Davis    2019      9112  -0.300481  1.299374  1.690053   
4          David Ortiz    2019       745   1.853656  1.147429  1.780034   
242      Nolan Arenado    2019      9777   1.387982  1.486992  1.420111   
316       Bryce Harper    2019     11579   0.409117  1.156018  1.915004   
..                 ...     ...       ...        ...       ...       ...   
306         Tyler Holt    2019     11391  -0.680623 -1.613994 -0.919386   
237     Shawn O'Malley    2019      9629  -0.870694 -1.455443 -1.369289   
384      Blake Swihart    2019     13176  -0.870694 -1.620600 -1.369289   
42         Carlos Ruiz    2019      2579   0.238053 -1.448837 -1.369289   
271      Rafael Ortega    2019     10323  -0.775658 -1.653631 -1.369289   

     weightSLG  weightAB  RBIsPredicted  
107   1.697391  1.381665     106.315379  
220   1.648640 

In [8]:
weightedRBIStats2019.reset_index()
dfa = weightedRBIStats2019.join(weightedAvgStats2019, how='outer', lsuffix='playerid', rsuffix='playerid')
dfa = dfa.join(weightedHRStats2019, how='outer', lsuffix='playerid', rsuffix='playerid')
dfa = dfa.join(weightedOPSStats2019, how='outer', lsuffix='playerid', rsuffix='playerid')
dfa = dfa.join(weightedRunStats2019, how='outer', lsuffix='playerid', rsuffix='playerid')
print(dfa)
dfa.to_csv('oldPredictions2019.csv')

            Nameplayerid  Seasonplayerid  playeridplayerid  weightAVG  \
0        Victor Martinez            2019               393  -0.015375   
1             Juan Uribe            2019               454  -1.599299   
2         Carlos Beltran            2019               589   0.123825   
3          Adrian Beltre            2019               639   0.979330   
4            David Ortiz            2019               745   1.853656   
..                   ...             ...               ...        ...   
500           Dae-Ho Lee            2019             18722  -0.110410   
501         Yuli Gurriel            2019             19198   1.169400   
502  Lourdes Gurriel Jr.            2019             19238   0.776587   
503        Shohei Ohtani            2019             19755   0.903301   
504            Juan Soto            2019             20123   1.125051   

     weightPAplayerid  weightSF  weightSLGplayerid  weightAB  RBIsPredicted  \
0            0.290596  0.925217          -0.